In [2]:
#read datafolder from Google Drive
from google.colab import drive
drive.mount('/content/drive')
data_folder = '/content/drive/My Drive/reuters21578/'


sgml_number_of_files = 21
sgml_file_name_template = 'reut2-NNN.sgm'

# Category files
category_files = {
    'to_': ('Topics', 'all-topics-strings.lc.txt'),
    'pl_': ('Places', 'all-places-strings.lc.txt'),
    'pe_': ('People', 'all-people-strings.lc.txt'),
    'or_': ('Organizations', 'all-orgs-strings.lc.txt'),
    'ex_': ('Exchanges', 'all-exchanges-strings.lc.txt')
}

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [0]:
# Create category dataframe

# Read all categories
category_data = []
category_dictionary={'Topics':[],'Places':[],'People':[],'Organizations':[],'Exchanges':[]}
for category_prefix in category_files.keys():
    with open(data_folder + category_files[category_prefix][1], 'r') as file:
        for category in file.readlines():
            category_data.append([category_prefix + category.strip().lower(), 
                                  category_files[category_prefix][0]])

# Create category dataframe
for i in category_data:
#     print(i[1])
    category_dictionary[i[1]].append(i[0].split('_')[1])
news_categories = pd.DataFrame(data=category_data)

# print "category_data: ", category_data
#(news_categories.values).tolist()

In [6]:
import re
import xml.sax.saxutils as saxutils
from bs4 import BeautifulSoup
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
lemmatizer = WordNetLemmatizer()

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;\\n]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def cleanUpSentence(r):#, stop_words = None#
    r = r.lower().replace("<br />", " ")
    r = REPLACE_BY_SPACE_RE.sub(' ', r)
    r = BAD_SYMBOLS_RE.sub('', r)

    r = ' '.join(word for word in r.split() if word not in STOPWORDS)

    words = word_tokenize(r)

    for w in words:
        w = lemmatizer.lemmatize(w)

    return r

In [0]:
# Parse SGML files
def strip_tags(text):
    return re.sub('<[^<]+?>', '', text).strip()

def unescape(text):
    return saxutils.unescape(text)
  
def makeDict(filename, document_X):
  with open(filename, 'rb') as file:

    content = BeautifulSoup(file.read().lower(),'html.parser')

    for newsline in content('reuters'):
      document_categories = []

      document_id = newsline['newid']
      document_body = strip_tags(str(newsline('text')[0].body)).replace('reuter\n&#3;', '')
      if document_body == 'None':
        continue

      doc_categories = strip_tags(str(newsline('topics')[0].body))
      doc_categories = unescape(doc_categories)

      document_body = unescape(document_body)

      document_X[document_id] = document_body

def readFiles(test_data = False):
  document_X = {}
  
  if test_data == True:
    file_name = sgml_file_name_template.replace('NNN', '021')
    filename = data_folder + file_name
    makeDict(filename, document_X)
  else:
    for i in range(sgml_number_of_files):
      if i < 10:
        seq = '00' + str(i)
      else:
        seq = '0' + str(i)

      file_name = sgml_file_name_template.replace('NNN', seq)
      print('Reading file: %s' % file_name)
      filename = data_folder + file_name
      makeDict(filename, document_X)
  return document_X



In [9]:
document_X= readFiles()

Reading file: reut2-000.sgm
Reading file: reut2-001.sgm
Reading file: reut2-002.sgm
Reading file: reut2-003.sgm
Reading file: reut2-004.sgm
Reading file: reut2-005.sgm
Reading file: reut2-006.sgm
Reading file: reut2-007.sgm
Reading file: reut2-008.sgm
Reading file: reut2-009.sgm
Reading file: reut2-010.sgm
Reading file: reut2-011.sgm
Reading file: reut2-012.sgm
Reading file: reut2-013.sgm
Reading file: reut2-014.sgm
Reading file: reut2-015.sgm
Reading file: reut2-016.sgm
Reading file: reut2-017.sgm
Reading file: reut2-018.sgm
Reading file: reut2-019.sgm
Reading file: reut2-020.sgm


In [10]:
# data preprocessing
import numpy as np

nltk.download('punkt')
nltk.download('wordnet')

def create_x_matrix(document_X):
    totalX = []
    for i, doc in document_X.items():
        totalX.append(cleanUpSentence(doc))
    max_vocab_size = 200
    input_tokenizer = Tokenizer(200)
    input_tokenizer.fit_on_texts(totalX)
    encoded_docs = input_tokenizer.texts_to_matrix(totalX, mode='count')
    return totalX,encoded_docs

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
totalX,encoded_docs=create_x_matrix(document_X)

In [0]:
#Create one-hot encode
def makeOneHotEncoding(totalX):
  words_in_body={}

  for i in range(len(totalX)):
      words=totalX[i].split(' ')
      words_in_body[i]=words    

  one_hot_label=[]
  for key,v in words_in_body.items():
      dict_temp={'Topics':0,'Places':0,'People':0,'Exchanges':0,'Organizations':0}
      for i in v:
          if i in category_dictionary['Topics']:
              dict_temp['Topics']+=1
          if i in category_dictionary['Places']:
              dict_temp['Places']+=1
          if i in category_dictionary['People']:
              dict_temp['People']+=1
          if i in category_dictionary['Exchanges']:
              dict_temp['Exchanges']+=1
          if i in category_dictionary['Organizations']:
              dict_temp['Organizations']+=1

      one_hot_label.append(dict_temp)


  one_hot_label_list = []
  for i in one_hot_label:

      one_hot_label_list.append(list(i.values()))
  return one_hot_label_list

In [0]:
one_hot_label_list = makeOneHotEncoding(totalX)

In [14]:
from keras.models import Sequential
from keras.layers import Dense,Flatten, Dropout,Embedding
nn = Sequential()
max_vocab_size = 200
nn.add(Embedding(200, 20, input_length=max_vocab_size))
nn.add(Dense(10, activation="relu", input_shape=(max_vocab_size,)))
nn.add(Dropout(0.15))
nn.add(Flatten())
nn.add(Dense(5,activation="softmax"))
nn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
nn.fit(np.array(encoded_docs), np.array(one_hot_label_list), batch_size=16, epochs=5,
          verbose=1, validation_split=0.2)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 14866 samples, validate on 3717 samples
Epoch 1/5
14866/14866 [==============================] - 4s 271us/step - loss: 3.1030 - acc: 0.8064 - val_loss: 3.1688 - val_acc: 0.7681
Epoch 2/5
14866/14866 [==============================] - 3s 217us/step - loss: 2.7796 - acc: 0.8147 - val_loss: 3.1070 - val_acc: 0.8036
Epoch 3/5
14866/14866 [==============================] - 3s 215us/step - loss: 2.7289 - acc: 0.8160 - val_loss: 3.0813 - val_acc: 0.7963
Epoch 4/5
14866/14866 [==============================] - 3s 217us/step - loss: 2.6849 - acc: 0.8144 - val_loss: 3.1077 - val_acc: 0.7926
Epoch 5/5
14866/14866 [==============================] - 3s 216us/step - loss: 2.6730 - acc: 0.8163 - val_loss: 3.1256 - val_acc: 0.8017


In [15]:
test_document_X = readFiles(test_data=True)
print(len(test_document_X))

460


In [16]:
test_total_X,test_encoded_X= create_x_matrix(test_document_X)
ground_truth_list = makeOneHotEncoding(test_total_X)
y = nn.predict(test_encoded_X)
print (y)

[[0.6975935  0.23532636 0.06036903 0.00425747 0.00245368]
 [0.21824951 0.5924399  0.04819081 0.0190319  0.12208779]
 [0.62695247 0.16869038 0.01963355 0.16197604 0.02274759]
 ...
 [0.6742106  0.21160336 0.05422151 0.03391149 0.02605302]
 [0.54931617 0.13644356 0.25580564 0.03680801 0.02162661]
 [0.8240941  0.11596978 0.04066919 0.00500179 0.01426519]]


In [0]:
def permissible(x, y):
  if (abs(x-y)) < 2:
    return True
  else:
    return False

def accuracy(predicted,actual):
    tp=0
    length = len(actual)
    for one_doc_idx in range(length):
        if permissible(predicted[one_doc_idx][0],actual[one_doc_idx][0]) and permissible(predicted[one_doc_idx][1],actual[one_doc_idx][1])\
          and permissible(predicted[one_doc_idx][2],actual[one_doc_idx][2]) and permissible(predicted[one_doc_idx][3],actual[one_doc_idx][3]):
            tp+=1
    return tp/float(length)
  

In [19]:
def getRankedOutput2(predicted_y):
    all_sorted_x = []
    for i in predicted_y:
        dict_temp={'Topics':i[0],'Places':i[1],'Peoples':i[2],'Exchanges':i[3],'Organizations':i[4]}
        sorted_x = sorted(dict_temp.items(), key=lambda kv: kv[1], reverse=True) 
        all_sorted_x.append(sorted_x)
    return all_sorted_x
   
print ("%f"%accuracy(y.tolist(), ground_truth_list))
predicted_ranked_output = getRankedOutput2(y)
actual_ranked_output=getRankedOutput2(np.array(ground_truth_list))

0.836957


In [20]:
pd.set_option('max_colwidth', 100)
my_df  = pd.DataFrame(columns = ['body', 'predicted ranking', 'actual ranking'])
my_df['body'] = test_total_X
my_df['predicted ranking'] = predicted_ranked_output
my_df['actual ranking'] = actual_ranked_output
my_df.head()

,body,predicted ranking,actual ranking
0,huge oil platforms dot gulf like beacons usually lit like christmas trees night one sitting astr...,"[(Topics, 0.6975935), (Places, 0.23532636), (Peoples, 0.060369026), (Exchanges, 0.0042574685), (...","[(Places, 12), (Topics, 3), (Peoples, 0), (Exchanges, 0), (Organizations, 0)]"
1,canadian auto workers union said accepted economic offer canadian division general motors corp g...,"[(Places, 0.5924399), (Topics, 0.21824951), (Organizations, 0.12208779), (Peoples, 0.04819081), ...","[(Places, 2), (Topics, 1), (Peoples, 0), (Exchanges, 0), (Organizations, 0)]"
2,canada development corp said polysar ltd unit completed refinancing package worth 830 mln canadi...,"[(Topics, 0.62695247), (Places, 0.16869038), (Exchanges, 0.16197604), (Organizations, 0.02274759...","[(Topics, 4), (Places, 1), (Peoples, 0), (Exchanges, 0), (Organizations, 0)]"
3,us attack iranian oil platform gulf monday appeared titfortat raid carefully orchestrated provoc...,"[(Topics, 0.55773973), (Places, 0.22359835), (Peoples, 0.18893665), (Organizations, 0.01862973),...","[(Places, 6), (Peoples, 2), (Topics, 0), (Exchanges, 0), (Organizations, 0)]"
4,brown disc products co inc unit fo genevar enterprises inc said purchased ongoing business trade...,"[(Topics, 0.5527812), (Exchanges, 0.26983488), (Places, 0.11818105), (Organizations, 0.029747142...","[(Topics, 0), (Places, 0), (Peoples, 0), (Exchanges, 0), (Organizations, 0)]"
